| Name | Type (Shape) | Description |
| --- | --- | --- |
| time | array (N,) | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (NY, NX, NZ) | X-meshgrid|
| Y | array (NY, NX, NZ) | Y-meshgrid|
| Z | array (NY, NX, NZ) | Y-meshgrid|
| Interpolant_u | RectangularGridInterpolator | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | RectangularGridInterpolator | Interpolant object for $ v(\mathbf{x}, t)  $ |
| Interpolant_w | RectangularGridInterpolator | Interpolant object for $ w(\mathbf{x}, t)  $ |
| periodic | list (4,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z <br /> periodic[3]: periodicity in time |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| Fmap | array (3,N) | integrated trajectory (=flow map) |
| dFdt | array (3,N) | velocity along trajectories (=time derivative of flow map) |
| DF | array (3,3,N) | gradient of flow map at every time-step |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import function to compute RK4 step
from ipynb.fs.defs.RK4_step import RK4_step

from tqdm.notebook import tqdm

In [4]:
def integration_dFdt(time, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady):
    
    # reshape x
    x = x.reshape(3, -1)
    
    # Initialize arrays for flow map and derivative of flow map
    Fmap = np.zeros((len(time), 3, x.shape[1]))
    dFdt = np.zeros((len(time)-1, 3, x.shape[1]))
    
    # Step-size
    dt = time[1]-time[0]
    
    counter = 0

    # initial conditions
    Fmap[counter,:,:] = x
    
    # Runge Kutta 4th order integration with fixed step size dt
    for t in tqdm(time[:-1]):
        
        Fmap[counter+1,:, :], dFdt[counter,:,:] = RK4_step(t, Fmap[counter,:, :], dt, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)
    
        counter += 1
    
    return Fmap, dFdt